## Chapter 3: Storage and Retrieval
Why should you, as an application developer, care how the database handles storage
and retrieval internally? You’re probably not going to implement your own storage
engine from scratch, but you do need to select a storage engine that is appropriate for
your application, from the many that are available. In order to tune a storage engine
to perform well on your kind of workload, you need to have a rough idea of what the
storage engine is doing under the hood.

There are two families of storage engines: **log-structured storage engines**, and **page-oriented storage engines**
such as B-trees.

---

### Log-structured storage engines
Storage engines that uses an append-only
sequence of records (called _log_). It doesn’t have to be human-readable; it might
be binary and intended only for other programs to read. 

This engine **is optimized for faster write** but has terrible performance if you have a large number of records in your database. to increase read performance databases uses an **index** wich is an additional structure that is derived from the primary data. Many databases
allow you to add and remove indexes, and this doesn’t affect the contents of the
database; it only affects the performance of queries.

#### Hash Indexes
Let’s say our data storage consists only of appending to a file, as in the preceding
example. Then the simplest possible indexing strategy is this: keep an in-memory
hash map where every key is mapped to a byte offset in the data file—the location at
which the value can be found. When you want to look up a value, use the hash map to find the offset in the
data file, seek to that location, and read the value.

![](images/image_6.png)

As described so far, we only ever append to a file—so how do we avoid eventually
running out of disk space? A good solution is to break the log into segments of a certain
size by closing a segment file when it reaches a certain size, and making subsequent
writes to a new segment file. We can then perform compaction on these
segments. Compaction means throwing away duplicate
keys in the log, and keeping only the most recent update for each key.

![](images/image_7.png)

**Limitations and issues in the real world**
* File format
    - It’s faster and simpler to use a binary format that first encodes the length of a string in bytes, followed by the raw string
* Deleting records
    - If you want to delete a key and its associated value, you have to append a special
deletion record to the data file (sometimes called a tombstone). When log segments
are merged, the tombstone tells the merging process to discard any previous
values for the deleted key.
* Crash recovery
    - If the database is restarted, the in-memory hash maps are lost. In principle, you
can restore each segment’s hash map by reading the entire segment file from
beginning to end and noting the offset of the most recent value for every key as
you go along. However, that might take a long time if the segment files are large,
which would make server restarts painful
* Partially written records
    - The database may crash at any time, including halfway through appending a
record to the log. Bitcask files include checksums, allowing such corrupted parts
of the log to be detected and ignored.
* Concurrency control
    - As writes are appended to the log in a strictly sequential order, a common implementation
choice is to have only one writer thread. Data file segments are
append-only and otherwise immutable, so they can be read concurrently by multiple
threads.
* The hash table must fit in memory, so if you have a very large number of keys,
you’re out of luck.
* Range queries are not efficient. For example, you cannot easily scan over all keys
between kitty00000 and kitty99999—you’d have to look up each key individually
in the hash maps.

#### SSTables and LSM-Trees
Now we can make a simple change to the format of our segment files: we require that
the sequence of key-value pairs is sorted by key. We call this format Sorted String Table, or **SSTable** for short.

**Advantages**
* Merging segments is simple and efficient, even if the files are bigger than the
available memory.

![](images/image_8.png)

* In order to find a particular key in the file, you no longer need to keep an index
of all the keys in memory.

Example: say you’re looking for
the key handiwork, but you don’t know the exact offset of that key in the segment
file. However, you do know the offsets for the keys handbag and handsome, and
because of the sorting you know that handiwork must appear between those two.
This means you can jump to the offset for handbag and scan from there until you
find handiwork (or not, if the key is not present in the file).

![](images/image_9.png)

* Since read requests need to scan over several key-value pairs in the requested
range anyway, it is possible to group those records into a block and compress it
before writing it to disk

**Read and write process**
* When a write comes in, add it to an in-memory balanced tree data structure (for
example, a red-black tree). This in-memory tree is sometimes called a memtable.
* When the memtable gets bigger than some threshold—typically a few megabytes
—write it out to disk as an SSTable file. This can be done efficiently because the
tree already maintains the key-value pairs sorted by key. The new SSTable file
becomes the most recent segment of the database. While the SSTable is being
written out to disk, writes can continue to a new memtable instance.
* In order to serve a read request, first try to find the key in the memtable, then in
the most recent on-disk segment, then in the next-older segment, etc.
* From time to time, run a merging and compaction process in the background to
combine segment files and to discard overwritten or deleted values.

Originally this indexing structure was described by Patrick O’Neil et al. under the
name _Log-Structured Merge-Tree_ (or **LSM-tree**).

---

### Page-oriented storage engines
#### B-Trees
The log-structured indexes we saw earlier break the database down into variable-size
segments, typically several megabytes or more in size, and always write a segment
sequentially. By contrast, B-trees break the database down into fixed-size blocks or
pages, traditionally 4 KB in size (sometimes bigger), and read or write one page at a
time. This design corresponds more closely to the underlying hardware, as disks are
also arranged in fixed-size blocks. 

Like SSTables, B-trees keep key-value pairs sorted by key, which allows efficient keyvalue
lookups and range queries. But that’s where the similarity ends.

![](images/image_10.png)

---

### Transaction Processing vs Analytics
An application
typically looks up a small number of records by some key, using an index. Records
are inserted or updated based on the user’s input. Because these applications are
interactive, the access pattern became known as **online transaction processing
(OLTP)**. However, databases also started being increasingly used for data analytics, which has
very different access patterns.In order
to differentiate this pattern of using databases from transaction processing, it has
been called **online analytic processing (OLAP)**.

#### Data Warehousing
A **data warehouse** is a separate database that analysts can query to their
hearts’ content, without affecting OLTP operations. The data warehouse contains
a read-only copy of the data in all the various OLTP systems in the company.
Data is extracted from OLTP databases (using either a periodic data dump or a continuous
stream of updates), transformed into an analysis-friendly schema, cleaned
up, and then loaded into the data warehouse. This process of getting data into the
warehouse is known as Extract–Transform–Load (ETL).

![](images/image_11.png)

#### Data warehouse data models
Many data warehouses
are used in a fairly formulaic style, known as a star schema (also known as dimensional
modeling, composed of a primary _fact table_ and related dimensions) and a variation of this template known as the snowflake schema, where dimensions are
further broken down into subdimensions.

![](images/image_12.png)

---

### Column-Oriented Storage
The idea behind column-oriented storage is simple: don’t store all the values from one
row together, but store all the values from each column together instead. If each column
is stored in a separate file, a query only needs to read and parse those columns
that are used in that query, which can save a lot of work.

![](images/image_13.png)
![](images/image_14.png)

#### Data Cubes and Materialized Views
OLAP systems are frequently used for aggregation queries (COUNT, SUM, AVG, MIN, MAX). One way of caching those queries is a **materialized view**. In a relational data model, it
is often defined like a standard (virtual) view: a table-like object whose contents are
the results of some query. The difference is that a materialized view is an actual copy
of the query results, written to disk, whereas a virtual view is just a shortcut for writing
queries.

A common special case of a materialized view is known as a data cube or OLAP cube. It is a grid of aggregates grouped by different dimensions.

![](images/image_15.png)